In [3]:
import pandas as pd
import numpy as np
import sklearn

In [4]:
training=pd.read_csv("twitter_training.csv",header=None)
training.shape

(24004, 4)

In [5]:
data_set=training[:2000]
data_set.shape
data_set.head()
print(data_set[2].unique())

['Positive' 'Neutral' 'Negative' 'Irrelevant']


In [6]:
data_set['encoded_text'] = data_set[2].astype('category').cat.codes
data_set.head()


<ipython-input-6-0645682098de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set['encoded_text'] = data_set[2].astype('category').cat.codes


,0,1,2,3,encoded_text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,3
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,3
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,3
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,3
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,3


In [7]:
data_set["encoded_text"].unique()

array([3, 2, 1, 0], dtype=int8)

In [8]:
import re

def replace_special_characters(input_string):
    # Define regex pattern to match special characters
    pattern = r"[^\w\s]"
    # Replace special characters with an empty string
    output_string = re.sub(pattern, "", input_string)
    return output_string
#replace_special_characters(str(x_test[2]))

for i in range(data_set.shape[0]):
    data_set[3][i]=replace_special_characters(str(data_set[3][i]))


<ipython-input-8-3c9dd5bd7872>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set[3][i]=replace_special_characters(str(data_set[3][i]))


In [9]:
from sklearn.model_selection import train_test_split
x_train,x,y_train,y = train_test_split(data_set[3],data_set["encoded_text"],test_size=0.5,random_state=0)
x_val,x_test,y_val,y_test = train_test_split(x,y,test_size=0.5,random_state=0)

In [10]:
x_train=x_train.to_list()
y_train=y_train.to_list()
x_val=x_val.to_list()
y_val=y_val.to_list()
x_test=x_test.to_list()
y_test=y_test.to_list()


In [11]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.4 MB/s eta 0:00:00


In [12]:
import transformers

In [13]:
from transformers import DistilBertTokenizerFast


In [14]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(list(x_train), truncation = True, padding = True  )

val_encodings = tokenizer(list(x_val), truncation = True, padding = True )
test_encodings = tokenizer(list(x_test), truncation = True, padding = True )

In [21]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))


val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [16]:
from transformers import TFDistilBertForSequenceClassification
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [17]:
from transformers import  TFTrainer, TFTrainingArguments


training_args = TFTrainingArguments(
    output_dir='./results',          
    num_train_epochs=7,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=1e-5,               
    logging_dir='./logs',            
    eval_steps=100                   
)

with training_args.strategy.scope():
    trainer_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 4 )


trainer = TFTrainer(
    model=trainer_model,                 
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_39', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [18]:
trainer.train()

In [19]:
trainer.evaluate()

{'eval_loss': 0.5098915696144104}

In [22]:
trainer.predict(test_dataset)
     

PredictionOutput(predictions=array([[ 4.1328125 , -1.5056742 , -1.4766762 , -2.1533847 ],
       [-2.1078684 , -2.6018999 , -0.67035556,  3.595404  ],
       [-3.0145614 , -2.4395063 ,  0.18581912,  3.4362588 ],
       ...,
       [-0.38434532,  0.7668893 ,  0.8655691 , -2.7985804 ],
       [-1.7673682 , -0.7842592 ,  0.5781198 ,  0.8605878 ],
       [-2.760887  , -1.5168747 ,  0.7994006 ,  2.0014985 ]],
      dtype=float32), label_ids=array([0, 3, 3, 2, 1, 2, 3, 3, 3, 3, 1, 3, 3, 1, 2, 3, 0, 3, 3, 2, 3, 3,
       1, 2, 0, 3, 2, 1, 3, 3, 2, 0, 1, 2, 2, 3, 2, 2, 0, 3, 2, 2, 2, 3,
       1, 3, 2, 3, 3, 3, 0, 2, 2, 2, 3, 0, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       2, 3, 2, 0, 1, 2, 3, 0, 3, 3, 2, 3, 3, 1, 2, 1, 2, 2, 3, 1, 3, 2,
       3, 0, 2, 1, 3, 3, 2, 3, 3, 3, 2, 2, 1, 3, 1, 2, 3, 1, 3, 3, 2, 1,
       2, 3, 0, 2, 2, 3, 2, 2, 3, 3, 1, 1, 0, 3, 1, 2, 3, 3, 2, 2, 2, 3,
       1, 3, 3, 3, 3, 2, 0, 1, 3, 3, 3, 3, 3, 0, 3, 2, 3, 3, 3, 3, 3, 2,
       3, 3, 1, 2, 3, 2, 2, 2, 1, 1, 0, 2, 2, 1, 

In [23]:
print(type(test_dataset))

<class 'tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset'>


In [39]:
output=trainer.predict(test_dataset)[0]
prediction_value = np.argmax(output[3])
prediction_value


2

In [48]:
p=np.zeros(output.shape[0])
for i in range(output.shape[0]):
  p[i]=np.argmax(output[i])
y=np.array(y_test)
p=p.astype(np.int32)


In [49]:

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y,p)
print("Accuracy:", accuracy)


Accuracy: 0.842
